<a href="https://colab.research.google.com/github/joezerr/Project/blob/main/ANN_Deep_Learning_no_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

referensi:
- https://towardsdatascience.com/a-guide-to-an-efficient-way-to-build-neural-network-architectures-part-i-hyper-parameter-8129009f131b
- https://towardsdatascience.com/pytorch-switching-to-the-gpu-a7c0b21e8a99
- https://neptune.ai/blog/hyperparameter-tuning-in-python-complete-guide
- https://www.tensorflow.org/tutorials/keras/keras_tuner
- https://stackoverflow.com/questions/68642923/typeerror-inputs-to-a-layer-should-be-tensors-got-prefetchdataset-shapes
- https://machinelearningmastery.com/how-to-improve-neural-network-stability-and-modeling-performance-with-data-scaling/
- udemy course

In [ ]:
import numpy as np
import math
import pandas as pd

**Import the dataset**

*dataset yang akan kita gunakan adalah dataset creditcard.csv*

In [ ]:
data = pd.read_csv('creditcard.csv')

In [ ]:
data.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,Class
0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,0.0
1,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,0.0
2,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.0
3,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,0.0
4,NaN,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,0.0


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8297 entries, 0 to 8296
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   V1      8295 non-null   float64
 1   V2      8292 non-null   float64
 2   V3      8292 non-null   float64
 3   V4      8290 non-null   float64
 4   V5      8290 non-null   float64
 5   V6      8295 non-null   float64
 6   V7      8292 non-null   float64
 7   V8      8292 non-null   float64
 8   V9      8291 non-null   float64
 9   V10     8293 non-null   float64
 10  Class   8296 non-null   float64
dtypes: float64(11)
memory usage: 713.1 KB


## **1a. Data Preprocessing**

In [ ]:
#split menjadi features(independent) dan target(dependent)
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

In [ ]:
print(X)

[[-1.35980713e+00 -7.27811730e-02  2.53634674e+00 ...  9.86979010e-02
   3.63786970e-01  9.07941720e-02]
 [ 1.19185711e+00  2.66150712e-01  1.66480113e-01 ...  8.51016550e-02
  -2.55425128e-01 -1.66974414e-01]
 [-1.35835406e+00 -1.34016307e+00  1.77320934e+00 ...  2.47675787e-01
  -1.51465432e+00  2.07642865e-01]
 ...
 [ 1.32592981e+00 -3.92763680e-02  1.84188710e-01 ...  1.14488460e-02
   1.47721040e+00 -3.71683168e-01]
 [ 1.37298315e+00 -3.82353569e-01  6.46021920e-02 ... -5.18603930e-02
   3.88560941e-01  9.51552000e-04]
 [-2.12548983e+00             nan             nan ...             nan
              nan             nan]]


In [ ]:
print(y)

[ 0.  0.  0. ...  0.  0. nan]


Langkah selanjutnya adalah check apakah ada masalah yang terdapat dalam dataset ini seperti null dll

In [ ]:
#check apakah ada null
data.isnull().sum()

V1       2
V2       5
V3       5
V4       7
V5       7
V6       2
V7       5
V8       5
V9       6
V10      4
Class    1
dtype: int64

seperti yang bisa kita lihat, terdapat masalah null values pada dataset kita. 

**Mengcheck variasi missing values selain (NaN)**

Kita tau bahwa terdapat missing values berupa (NaN), akan tetapi, ada kemungkinan terdapat missing values dalam bentuk yang lain seperti N/a, na, maupun np.nan. Untuk mengatasinya, saya akan membuat variable yang menampung bentuk missing values yang mungkin tidak terdeteksi kemudian membaca ulang datasetnya dengan menyertakan missing values tambahan.

In [ ]:
path = '/content/creditcard.csv'

In [ ]:
missing_vals = ['N/a', 'na', np.nan]
data = pd.read_csv(path, na_values=missing_vals)

In [ ]:
#check apakah ada null
data.isnull().sum()

V1       2
V2       4
V3       4
V4       6
V5       7
V6       2
V7       5
V8       5
V9       6
V10      4
Class    1
dtype: int64

ternyata, tidak ada missing values dalam bentuk lain yang berarti hanya terdapat missing values dalam bentuk NaN

**Menghilangkan missing_values (NaN)**

values yang kosong akan saya isi dengan mediannya daripada di drop, alasannya adalah jika value null tersebut didrop, akan timbul kemungkinan kehilangan banyak data dan juga bias tertentu 

In [ ]:
print(X)

[[-1.35980713e+00 -7.27811730e-02  2.53634674e+00 ...  9.86979010e-02
   3.63786970e-01  9.07941720e-02]
 [ 1.19185711e+00  2.66150712e-01  1.66480113e-01 ...  8.51016550e-02
  -2.55425128e-01 -1.66974414e-01]
 [-1.35835406e+00 -1.34016307e+00  1.77320934e+00 ...  2.47675787e-01
  -1.51465432e+00  2.07642865e-01]
 ...
 [ 1.32592981e+00 -3.92763680e-02  1.84188710e-01 ...  1.14488460e-02
   1.47721040e+00 -3.71683168e-01]
 [ 1.37298315e+00 -3.82353569e-01  6.46021920e-02 ... -5.18603930e-02
   3.88560941e-01  9.51552000e-04]
 [-2.12548983e+00             nan             nan ...             nan
              nan             nan]]


Null values (missing values) yang ada akan diisi dengan mediannya. Median dipilih dibandingkan dengan mean karena penggunaan mean mempunyai kemungkinan terdapat outlier. Jadi, median digunakan untuk memperkecil kemungkinan adanya outlier

In [ ]:
from sklearn.impute import SimpleImputer
imp = SimpleImputer(missing_values=np.nan,strategy='median',verbose=0)

In [ ]:
imp = imp.fit(X[:,:])
X[:,:] = imp.transform(X[:,:])

In [ ]:
print(X)

[[-1.35980713e+00 -7.27811730e-02  2.53634674e+00 ...  9.86979010e-02
   3.63786970e-01  9.07941720e-02]
 [ 1.19185711e+00  2.66150712e-01  1.66480113e-01 ...  8.51016550e-02
  -2.55425128e-01 -1.66974414e-01]
 [-1.35835406e+00 -1.34016307e+00  1.77320934e+00 ...  2.47675787e-01
  -1.51465432e+00  2.07642865e-01]
 ...
 [ 1.32592981e+00 -3.92763680e-02  1.84188710e-01 ...  1.14488460e-02
   1.47721040e+00 -3.71683168e-01]
 [ 1.37298315e+00 -3.82353569e-01  6.46021920e-02 ... -5.18603930e-02
   3.88560941e-01  9.51552000e-04]
 [-2.12548983e+00  3.14873657e-01  9.39410571e-01 ...  9.93581950e-03
   6.60369234e-01 -2.86126809e-01]]


In [ ]:
#check kembali apakah masih terdapat null
np.isnan(X).any()

False

null values sudah diisi dengan median dan sudah tidak ada null values lagi

saya tidak akan melakukan hot-encoding karena tidak terdapat categorical features pada datasetnya

# **1b. Split dataset menjadi train, test, val**

In [ ]:
#pertama split data menjadi training dan data sisa
from sklearn.model_selection import train_test_split
X_train, X_rem, y_train, y_rem = train_test_split(X, y, train_size=0.8, random_state=1)

In [ ]:
#define val test = 0.5 (setengah dari data sisa)
test_size=0.5
X_valid, X_test, y_valid, y_test = train_test_split(X_rem, y_rem, test_size=0.5, random_state=1)

In [ ]:
print(X_train.shape), print(y_train.shape)
print(X_valid.shape), print(y_valid.shape)
print(X_test.shape), print(y_test.shape)

(6637, 10)
(6637,)
(830, 10)
(830,)
(830, 10)
(830,)


(None, None)

**Feature Scalling**

saya akan menggunakan salah satu jenis feature scalling yaitu Standard Scaller

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_valid = sc.transform(X_valid)
X_test = sc.transform(X_test)

In [ ]:
print(np.any(np.isnan(X_test)))
print(np.any(np.isnan(y_test)))

False
False


In [ ]:
print(X_train)

[[ 1.01209597  0.09467242 -0.18117219 ... -0.07907141  0.65856756
  -0.22474287]
 [-0.57951486  0.07603959  0.63431434 ...  0.92224636 -1.05001121
   1.19361436]
 [-0.95563935 -0.96193615  0.96955175 ...  0.41513011  0.65434602
  -1.66837416]
 ...
 [ 0.92500699 -0.11188883 -0.10323252 ...  0.06296954 -0.22848967
  -0.00287492]
 [ 1.04399108 -0.09431792 -1.89455017 ...  0.5969556   0.45603856
  -0.38149333]
 [-2.12776158 -2.45298926  0.68648194 ...  0.1701497   0.0093062
  -0.36065286]]


In [ ]:
print(X_test)

[[ 0.83797129 -0.78392556 -1.37269714 ... -0.03941315  0.10987524
  -0.701136  ]
 [ 1.0271177  -0.29604599 -0.87963249 ... -0.07192835  0.37618621
  -0.71093434]
 [ 0.38515244  0.4279249  -0.08079038 ... -0.73379922  0.37680409
  -0.63423277]
 ...
 [ 0.67705212 -0.50521265 -0.69496446 ... -0.22183499  0.51066579
  -0.27013325]
 [ 1.12449727 -0.43923091 -0.50757889 ... -0.19544833 -0.25864876
   0.39490973]
 [ 0.89865829 -0.41945029 -0.43123514 ... -0.01140213 -0.03664804
  -0.01713616]]


# **1c. Base arsitektur**

*Saya akan membuat arsitektur ANN* 

### **ANN**

### inisialiasi ANN

In [ ]:
import tensorflow as tf #backpropagation
annmodel = tf.keras.models.Sequential()

#### Buat baseline arsitektur

*n nodes input layer, 2 buah*
*hidden layer =  𝟐 × 𝒏 nodes awal dan layer akhir banyak kelas nya*
*(𝒏, 𝟐 × 𝒏, 𝟐 × 𝒏, 𝒏𝒖𝒎_𝒄𝒍𝒂𝒔𝒔). Activation function = ReLU.*

input layer

In [ ]:
annmodel.add(tf.keras.layers.Dense(units=10, activation='relu'))

*hidden layer pertama*

In [ ]:
annmodel.add(tf.keras.layers.Dense(units=20, activation='relu'))

*hidden layar kedua*

In [ ]:
annmodel.add(tf.keras.layers.Dense(units=20, activation='relu'))

*output layer*

*activation function untuk output layer terakhir adalah sigmoid karena hasil yang diinginkan adalah true or false / fraud or not fraud*

In [ ]:
annmodel.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

In [ ]:
np.isnan(X_train).any()

False

In [ ]:
np.isnan(y_train).any()

True

###Train ANN

*Compile ann*

In [ ]:
from tensorflow.python.ops.clip_ops import clip_by_global_norm
opt = tf.keras.optimizers.Adam(learning_rate=0.001) #clip_by_global_norm=

In [ ]:
annmodel.compile(optimizer= 'adam', loss= 'binary_crossentropy', metrics= ['accuracy'])

In [ ]:
annmodel.fit(X_train, y_train, batch_size=128, epochs=100)

Epoch 1/100
52/52 [==============================] - 3s 3ms/step - loss: nan - accuracy: 0.9797
Epoch 2/100
52/52 [==============================] - 0s 3ms/step - loss: nan - accuracy: 0.9968
Epoch 3/100
52/52 [==============================] - 0s 3ms/step - loss: nan - accuracy: 0.9968
Epoch 4/100
52/52 [==============================] - 0s 3ms/step - loss: nan - accuracy: 0.9968
Epoch 5/100
52/52 [==============================] - 0s 3ms/step - loss: nan - accuracy: 0.9968
Epoch 6/100
52/52 [==============================] - 0s 3ms/step - loss: nan - accuracy: 0.9968
Epoch 7/100
52/52 [==============================] - 0s 3ms/step - loss: nan - accuracy: 0.9968
Epoch 8/100
52/52 [==============================] - 0s 3ms/step - loss: nan - accuracy: 0.9968
Epoch 9/100
52/52 [==============================] - 0s 3ms/step - loss: nan - accuracy: 0.9968
Epoch 10/100
52/52 [==============================] - 0s 3ms/step - loss: nan - accuracy: 0.9968
Epoch 11/100
52/52 [===================

seperti yang bisa kita lihat diatas, mulai dari epoch 1, akurasi yang dihasilkan sudah mencapai 99% dan validation loss yang dihasilkan adalah nan. Nan berarti validation mempunyai loss yang sangat besar (keluar dari range 0-1). Hal ini menandakan adanya overfitting pada saat mentraining data kita.

Penyebabnya adalah ketika menggunakan cross-entropy loss untuk klasifikasi, prediksi yang jelek akan mendapatkan punishment yang lebih kuat dibandigkan dengan prediksi yang bagus. Ini berarti ketika kita mempunyai banyak image dengan loss yang rendah namun terdapat satu image lain yang mempunyai loss yang sangat tinggi, maka loss validation kita akan tiba-tiba spiking-up. Hasil yang ditunjukan pada awal epoch ke-1 yaitu accuracy dan val loss kedua-duanya naik menyebabkan overfit. Hal ini menyebabkan network hanya belajar pada pola tertentu yang relevan untuk training set dan kurang baik untuk generalisasi. Namun disaat yang bersamaan, network tetap mempelajari pattern yang berguna untuk generalisasi karena makin banyak image untuk diklasifikasi

referensi: - https://stats.stackexchange.com/questions/282160/how-is-it-possible-that-validation-loss-is-increasing-while-validation-accuracy

# 1d. modifikasi arsitektur

pendekatan modifikasi yang saya pilih adalah hyperparameter tuning. hyperparameter tuning adalah proses test untuk tuning konfigurasi hyperparameter yang berbeda-beda pada saat training data dan memilih yang terbaik

alasan saya memilih hyperparameter tuning khususnya batchnormalization dan dropout adalah:

Penambahan layer dropout mengatasi masalah overfitting yang ada. Overfitting disebabkan oleh adanya pattern palsu yang dipelajari network dalam training data. Untuk mengenali pola ini, weight dijadikan patokan untuk mengukur. Dropout layer berfungsi untuk mendrop sedikit random layer unit pada saat training. Hal ini membuat pattern yang palsu lebih sulit dipelajari oleh network pada saat mentraining data sehingga network dipaksa untuk mencari patternyang lebih luas, umum, dan mempunyai bobot yang kuat.

Selain dropout layer, penambahan batch normalization dilakukan untuk menormalisasi data didalam layer network. Batch_normalization dilakukan pada setiap kali batch datang. Hal pertama yang dilakukan adalah menormalisasi dengan mean dan standard deviation masing-masing, kemudian juga akan membuat data memiliki scale yang baru dengan dua trainable rescaling parameter.

referensi: -https://www.kaggle.com/code/ryanholbrook/dropout-and-batch-normalization

##Retrain ann

##percobaan ke-1

In [ ]:
import tensorflow as tf #backpropagation
annmodelrework = tf.keras.models.Sequential()

In [ ]:
annmodelrework.add(tf.keras.layers.Dense(units=10, activation='relu'))
annmodelrework.add(tf.keras.layers.BatchNormalization())
annmodelrework.add(tf.keras.layers.Dropout(0.4))

In [ ]:
annmodelrework.add(tf.keras.layers.Dense(units=20, activation='relu'))
annmodelrework.add(tf.keras.layers.BatchNormalization())
annmodelrework.add(tf.keras.layers.Dropout(0.4))

In [ ]:
annmodelrework.add(tf.keras.layers.Dense(units=20, activation='relu'))
annmodelrework.add(tf.keras.layers.BatchNormalization())
annmodelrework.add(tf.keras.layers.Dropout(0.4))

In [ ]:
annmodelrework.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

In [ ]:
annmodelrework.fit(X_train, y_train, batch_size=128, epochs=100)

Epoch 1/100
52/52 [==============================] - 1s 5ms/step - loss: nan - accuracy: 0.6387
Epoch 2/100
52/52 [==============================] - 0s 5ms/step - loss: nan - accuracy: 0.9968
Epoch 3/100
52/52 [==============================] - 0s 5ms/step - loss: nan - accuracy: 0.9968
Epoch 4/100
52/52 [==============================] - 0s 5ms/step - loss: nan - accuracy: 0.9968
Epoch 5/100
52/52 [==============================] - 0s 5ms/step - loss: nan - accuracy: 0.9968
Epoch 6/100
52/52 [==============================] - 0s 5ms/step - loss: nan - accuracy: 0.9968
Epoch 7/100
52/52 [==============================] - 0s 5ms/step - loss: nan - accuracy: 0.9968
Epoch 8/100
52/52 [==============================] - 0s 5ms/step - loss: nan - accuracy: 0.9968
Epoch 9/100
52/52 [==============================] - 0s 5ms/step - loss: nan - accuracy: 0.9968
Epoch 10/100
52/52 [==============================] - 0s 5ms/step - loss: nan - accuracy: 0.9968
Epoch 11/100
52/52 [===================

sepertinya nilai loss masih tetap nan dan akurasi masih 0.99.
sesuaikan kembali hyperparameternya

##percobaan ke-2

In [ ]:
#ganti dropout rate ke 0.3
annmodelrework.add(tf.keras.layers.Dense(units=10, activation='relu'))
annmodelrework.add(tf.keras.layers.BatchNormalization())
annmodelrework.add(tf.keras.layers.Dropout(0.3))
annmodelrework.add(tf.keras.layers.Dense(units=20, activation='relu'))
annmodelrework.add(tf.keras.layers.BatchNormalization())
annmodelrework.add(tf.keras.layers.Dropout(0.3))
annmodelrework.add(tf.keras.layers.Dense(units=20, activation='relu'))
annmodelrework.add(tf.keras.layers.BatchNormalization())
annmodelrework.add(tf.keras.layers.Dropout(0.3))
annmodelrework.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

In [ ]:
annmodelrework.fit(X_train, y_train, batch_size=128, epochs=100)

Epoch 1/100
52/52 [==============================] - 0s 5ms/step - loss: nan - accuracy: 0.9968
Epoch 2/100
52/52 [==============================] - 0s 5ms/step - loss: nan - accuracy: 0.9968
Epoch 3/100
52/52 [==============================] - 0s 5ms/step - loss: nan - accuracy: 0.9968
Epoch 4/100
52/52 [==============================] - 0s 5ms/step - loss: nan - accuracy: 0.9968
Epoch 5/100
52/52 [==============================] - 0s 5ms/step - loss: nan - accuracy: 0.9968
Epoch 6/100
52/52 [==============================] - 0s 5ms/step - loss: nan - accuracy: 0.9968
Epoch 7/100
52/52 [==============================] - 0s 5ms/step - loss: nan - accuracy: 0.9968
Epoch 8/100
52/52 [==============================] - 0s 5ms/step - loss: nan - accuracy: 0.9968
Epoch 9/100
52/52 [==============================] - 0s 5ms/step - loss: nan - accuracy: 0.9968
Epoch 10/100
52/52 [==============================] - 0s 5ms/step - loss: nan - accuracy: 0.9968
Epoch 11/100
52/52 [===================

sepertinya nilai loss masih tetap nan dan akurasi masih 0.99.
sesuaikan kembali **hyperparameternya**

##percobaan ke-3

In [ ]:
annmodelrework.add(tf.keras.layers.Dense(units=10, activation='relu'))
annmodelrework.add(tf.keras.layers.BatchNormalization())
annmodelrework.add(tf.keras.layers.Dropout(0.25))
annmodelrework.add(tf.keras.layers.Dense(units=20, activation='relu'))
annmodelrework.add(tf.keras.layers.BatchNormalization())
annmodelrework.add(tf.keras.layers.Dropout(0.25))
annmodelrework.add(tf.keras.layers.Dense(units=20, activation='relu'))
annmodelrework.add(tf.keras.layers.BatchNormalization())
annmodelrework.add(tf.keras.layers.Dropout(0.25))
annmodelrework.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

In [ ]:
annmodelrework.fit(X_train, y_train, batch_size=128, epochs=100)

Epoch 1/100
52/52 [==============================] - 4s 11ms/step - loss: nan - accuracy: 0.9968
Epoch 2/100
52/52 [==============================] - 1s 12ms/step - loss: nan - accuracy: 0.9968
Epoch 3/100
52/52 [==============================] - 1s 12ms/step - loss: nan - accuracy: 0.9968
Epoch 4/100
52/52 [==============================] - 1s 11ms/step - loss: nan - accuracy: 0.9968
Epoch 5/100
52/52 [==============================] - 1s 11ms/step - loss: nan - accuracy: 0.9968
Epoch 6/100
52/52 [==============================] - 1s 11ms/step - loss: nan - accuracy: 0.9968
Epoch 7/100
52/52 [==============================] - 1s 12ms/step - loss: nan - accuracy: 0.9968
Epoch 8/100
52/52 [==============================] - 1s 12ms/step - loss: nan - accuracy: 0.9968
Epoch 9/100
52/52 [==============================] - 1s 12ms/step - loss: nan - accuracy: 0.9968
Epoch 10/100
52/52 [==============================] - 1s 12ms/step - loss: nan - accuracy: 0.9968
Epoch 11/100
52/52 [=========

sepertinya nilai loss masih tetap nan dan akurasi masih 0.99.
sesuaikan kembali **hyperparameternya**

##percobaan ke-4

In [ ]:
annmodelrework.add(tf.keras.layers.Dense(units=10, activation='relu'))
annmodelrework.add(tf.keras.layers.BatchNormalization())
annmodelrework.add(tf.keras.layers.Dropout(0.5))
annmodelrework.add(tf.keras.layers.Dense(units=20, activation='relu'))
annmodelrework.add(tf.keras.layers.BatchNormalization())
annmodelrework.add(tf.keras.layers.Dropout(0.5))
annmodelrework.add(tf.keras.layers.Dense(units=20, activation='relu'))
annmodelrework.add(tf.keras.layers.BatchNormalization())
annmodelrework.add(tf.keras.layers.Dropout(0.5))
annmodelrework.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

In [ ]:
annmodelrework.fit(X_train, y_train, batch_size=128, epochs=100)

Epoch 1/100
52/52 [==============================] - 1s 12ms/step - loss: nan - accuracy: 0.9968
Epoch 2/100
52/52 [==============================] - 1s 11ms/step - loss: nan - accuracy: 0.9968
Epoch 3/100
52/52 [==============================] - 1s 21ms/step - loss: nan - accuracy: 0.9968
Epoch 4/100
52/52 [==============================] - 1s 12ms/step - loss: nan - accuracy: 0.9968
Epoch 5/100
52/52 [==============================] - 1s 12ms/step - loss: nan - accuracy: 0.9968
Epoch 6/100
52/52 [==============================] - 1s 12ms/step - loss: nan - accuracy: 0.9968
Epoch 7/100
52/52 [==============================] - 1s 12ms/step - loss: nan - accuracy: 0.9968
Epoch 8/100
52/52 [==============================] - 1s 12ms/step - loss: nan - accuracy: 0.9968
Epoch 9/100
52/52 [==============================] - 1s 12ms/step - loss: nan - accuracy: 0.9968
Epoch 10/100
52/52 [==============================] - 1s 12ms/step - loss: nan - accuracy: 0.9968
Epoch 11/100
52/52 [=========

sepertinya nilai loss masih tetap nan dan akurasi masih 0.99.
sesuaikan kembali **hyperparameternya**

##percobaan ke -5

In [ ]:
annmodelrework.add(tf.keras.layers.Dense(units=10, activation='relu'))
annmodelrework.add(tf.keras.layers.BatchNormalization())
annmodelrework.add(tf.keras.layers.Dropout(0.2))
annmodelrework.add(tf.keras.layers.Dense(units=20, activation='relu'))
annmodelrework.add(tf.keras.layers.BatchNormalization())
annmodelrework.add(tf.keras.layers.Dropout(0.2))
annmodelrework.add(tf.keras.layers.Dense(units=20, activation='relu'))
annmodelrework.add(tf.keras.layers.BatchNormalization())
annmodelrework.add(tf.keras.layers.Dropout(0.2))
annmodelrework.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

In [ ]:
annmodelrework.fit(X_train, y_train, batch_size=128, epochs=100)

Epoch 1/100
52/52 [==============================] - 1s 11ms/step - loss: nan - accuracy: 0.9968
Epoch 2/100
52/52 [==============================] - 1s 12ms/step - loss: nan - accuracy: 0.9968
Epoch 3/100
52/52 [==============================] - 1s 12ms/step - loss: nan - accuracy: 0.9968
Epoch 4/100
52/52 [==============================] - 1s 12ms/step - loss: nan - accuracy: 0.9968
Epoch 5/100
52/52 [==============================] - 1s 12ms/step - loss: nan - accuracy: 0.9968
Epoch 6/100
52/52 [==============================] - 1s 12ms/step - loss: nan - accuracy: 0.9968
Epoch 7/100
52/52 [==============================] - 1s 12ms/step - loss: nan - accuracy: 0.9968
Epoch 8/100
52/52 [==============================] - 1s 12ms/step - loss: nan - accuracy: 0.9968
Epoch 9/100
52/52 [==============================] - 1s 12ms/step - loss: nan - accuracy: 0.9968
Epoch 10/100
52/52 [==============================] - 1s 12ms/step - loss: nan - accuracy: 0.9968
Epoch 11/100
52/52 [=========

sepertinya nilai loss masih tetap nan dan akurasi masih 0.99. sesuaikan kembali hyperparameternya

##percobaan ke-6

In [ ]:
annmodelrework.add(tf.keras.layers.Dense(units=10, activation='relu'))
annmodelrework.add(tf.keras.layers.BatchNormalization())
annmodelrework.add(tf.keras.layers.Dropout(0.2))
annmodelrework.add(tf.keras.layers.Dense(units=20, activation='relu'))
annmodelrework.add(tf.keras.layers.BatchNormalization())
annmodelrework.add(tf.keras.layers.Dropout(0.2))
annmodelrework.add(tf.keras.layers.Dense(units=30, activation='relu'))
annmodelrework.add(tf.keras.layers.BatchNormalization())
annmodelrework.add(tf.keras.layers.Dropout(0.2))
annmodelrework.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

In [ ]:
annmodelrework.fit(X_train, y_train, batch_size=128, epochs=100)

Epoch 1/100
52/52 [==============================] - 1s 12ms/step - loss: nan - accuracy: 0.9968
Epoch 2/100
52/52 [==============================] - 1s 12ms/step - loss: nan - accuracy: 0.9968
Epoch 3/100
52/52 [==============================] - 1s 14ms/step - loss: nan - accuracy: 0.9968
Epoch 4/100
52/52 [==============================] - 1s 12ms/step - loss: nan - accuracy: 0.9968
Epoch 5/100
52/52 [==============================] - 1s 12ms/step - loss: nan - accuracy: 0.9968
Epoch 6/100
52/52 [==============================] - 1s 12ms/step - loss: nan - accuracy: 0.9968
Epoch 7/100
52/52 [==============================] - 1s 12ms/step - loss: nan - accuracy: 0.9968
Epoch 8/100
52/52 [==============================] - 1s 12ms/step - loss: nan - accuracy: 0.9968
Epoch 9/100
52/52 [==============================] - 1s 12ms/step - loss: nan - accuracy: 0.9968
Epoch 10/100
52/52 [==============================] - 1s 12ms/step - loss: nan - accuracy: 0.9968
Epoch 11/100
52/52 [=========

sepertinya nilai loss masih tetap nan dan akurasi masih 0.99. sesuaikan kembali hyperparameternya

##percobaan ke-7

In [ ]:
annmodelrework.add(tf.keras.layers.Dense(units=20, activation='relu'))
annmodelrework.add(tf.keras.layers.BatchNormalization())
annmodelrework.add(tf.keras.layers.Dropout(0.2))
annmodelrework.add(tf.keras.layers.Dense(units=30, activation='relu'))
annmodelrework.add(tf.keras.layers.BatchNormalization())
annmodelrework.add(tf.keras.layers.Dropout(0.2))
annmodelrework.add(tf.keras.layers.Dense(units=40, activation='relu'))
annmodelrework.add(tf.keras.layers.BatchNormalization())
annmodelrework.add(tf.keras.layers.Dropout(0.2))
annmodelrework.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

In [ ]:
annmodelrework.fit(X_train, y_train, batch_size=128, epochs=100)

Epoch 1/100
52/52 [==============================] - 1s 11ms/step - loss: nan - accuracy: 0.9968
Epoch 2/100
52/52 [==============================] - 1s 12ms/step - loss: nan - accuracy: 0.9968
Epoch 3/100
52/52 [==============================] - 1s 11ms/step - loss: nan - accuracy: 0.9968
Epoch 4/100
52/52 [==============================] - 1s 12ms/step - loss: nan - accuracy: 0.9968
Epoch 5/100
52/52 [==============================] - 1s 12ms/step - loss: nan - accuracy: 0.9968
Epoch 6/100
52/52 [==============================] - 1s 12ms/step - loss: nan - accuracy: 0.9968
Epoch 7/100
52/52 [==============================] - 1s 12ms/step - loss: nan - accuracy: 0.9968
Epoch 8/100
52/52 [==============================] - 1s 11ms/step - loss: nan - accuracy: 0.9968
Epoch 9/100
52/52 [==============================] - 1s 12ms/step - loss: nan - accuracy: 0.9968
Epoch 10/100
52/52 [==============================] - 1s 12ms/step - loss: nan - accuracy: 0.9968
Epoch 11/100
52/52 [=========

sepertinya nilai loss masih tetap nan dan akurasi masih 0.99. sesuaikan kembali hyperparameternya

## percobaan ke - 8

In [ ]:
annmodelrework.add(tf.keras.layers.Dense(units=20, activation='relu'))
annmodelrework.add(tf.keras.layers.BatchNormalization())
annmodelrework.add(tf.keras.layers.Dropout(0.2))
annmodelrework.add(tf.keras.layers.Dense(units=40, activation='relu'))
annmodelrework.add(tf.keras.layers.BatchNormalization())
annmodelrework.add(tf.keras.layers.Dropout(0.2))
annmodelrework.add(tf.keras.layers.Dense(units=40, activation='relu'))
annmodelrework.add(tf.keras.layers.BatchNormalization())
annmodelrework.add(tf.keras.layers.Dropout(0.2))
annmodelrework.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

In [ ]:
annmodelrework.fit(X_train, y_train, batch_size=128, epochs=100)

Epoch 1/100
52/52 [==============================] - 1s 12ms/step - loss: nan - accuracy: 0.9968
Epoch 2/100
52/52 [==============================] - 1s 12ms/step - loss: nan - accuracy: 0.9968
Epoch 3/100
52/52 [==============================] - 1s 12ms/step - loss: nan - accuracy: 0.9968
Epoch 4/100
52/52 [==============================] - 1s 12ms/step - loss: nan - accuracy: 0.9968
Epoch 5/100
52/52 [==============================] - 1s 12ms/step - loss: nan - accuracy: 0.9968
Epoch 6/100
52/52 [==============================] - 1s 12ms/step - loss: nan - accuracy: 0.9968
Epoch 7/100
52/52 [==============================] - 1s 12ms/step - loss: nan - accuracy: 0.9968
Epoch 8/100
52/52 [==============================] - 1s 12ms/step - loss: nan - accuracy: 0.9968
Epoch 9/100
52/52 [==============================] - 1s 12ms/step - loss: nan - accuracy: 0.9968
Epoch 10/100
52/52 [==============================] - 1s 12ms/step - loss: nan - accuracy: 0.9968
Epoch 11/100
52/52 [=========

sepertinya nilai loss masih tetap nan dan akurasi masih 0.99. sesuaikan kembali hyperparameternya

## percobaan ke-9

In [ ]:
annmodelrework.add(tf.keras.layers.Dense(units=40, activation='relu'))
annmodelrework.add(tf.keras.layers.BatchNormalization())
annmodelrework.add(tf.keras.layers.Dropout(0.2))
annmodelrework.add(tf.keras.layers.Dense(units=30, activation='relu'))
annmodelrework.add(tf.keras.layers.BatchNormalization())
annmodelrework.add(tf.keras.layers.Dropout(0.2))
annmodelrework.add(tf.keras.layers.Dense(units=20, activation='relu'))
annmodelrework.add(tf.keras.layers.BatchNormalization())
annmodelrework.add(tf.keras.layers.Dropout(0.2))
annmodelrework.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

In [ ]:
annmodelrework.fit(X_train, y_train, batch_size=128, epochs=100)

Epoch 1/100
52/52 [==============================] - 1s 11ms/step - loss: nan - accuracy: 0.9968
Epoch 2/100
52/52 [==============================] - 1s 12ms/step - loss: nan - accuracy: 0.9968
Epoch 3/100
52/52 [==============================] - 1s 12ms/step - loss: nan - accuracy: 0.9968
Epoch 4/100
52/52 [==============================] - 1s 12ms/step - loss: nan - accuracy: 0.9968
Epoch 5/100
52/52 [==============================] - 1s 12ms/step - loss: nan - accuracy: 0.9968
Epoch 6/100
52/52 [==============================] - 1s 12ms/step - loss: nan - accuracy: 0.9968
Epoch 7/100
52/52 [==============================] - 1s 13ms/step - loss: nan - accuracy: 0.9968
Epoch 8/100
52/52 [==============================] - 1s 12ms/step - loss: nan - accuracy: 0.9968
Epoch 9/100
52/52 [==============================] - 1s 12ms/step - loss: nan - accuracy: 0.9968
Epoch 10/100
52/52 [==============================] - 1s 12ms/step - loss: nan - accuracy: 0.9968
Epoch 11/100
52/52 [=========

sepertinya nilai loss masih tetap nan dan akurasi masih 0.99. sesuaikan kembali hyperparameternya

## percobaan ke-10

In [ ]:
annmodelrework.add(tf.keras.layers.Dense(units=30, activation='relu'))
annmodelrework.add(tf.keras.layers.BatchNormalization())
annmodelrework.add(tf.keras.layers.Dropout(0.2))
annmodelrework.add(tf.keras.layers.Dense(units=30, activation='relu'))
annmodelrework.add(tf.keras.layers.BatchNormalization())
annmodelrework.add(tf.keras.layers.Dropout(0.2))
annmodelrework.add(tf.keras.layers.Dense(units=30, activation='relu'))
annmodelrework.add(tf.keras.layers.BatchNormalization())
annmodelrework.add(tf.keras.layers.Dropout(0.2))
annmodelrework.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

In [ ]:
annmodelrework.fit(X_train, y_train, batch_size=128, epochs=100)

Epoch 1/100
52/52 [==============================] - 1s 12ms/step - loss: nan - accuracy: 0.9968
Epoch 2/100
52/52 [==============================] - 1s 12ms/step - loss: nan - accuracy: 0.9968
Epoch 3/100
52/52 [==============================] - 1s 12ms/step - loss: nan - accuracy: 0.9968
Epoch 4/100
52/52 [==============================] - 1s 13ms/step - loss: nan - accuracy: 0.9968
Epoch 5/100
52/52 [==============================] - 1s 12ms/step - loss: nan - accuracy: 0.9968
Epoch 6/100
52/52 [==============================] - 1s 12ms/step - loss: nan - accuracy: 0.9968
Epoch 7/100
52/52 [==============================] - 1s 13ms/step - loss: nan - accuracy: 0.9968
Epoch 8/100
52/52 [==============================] - 1s 17ms/step - loss: nan - accuracy: 0.9968
Epoch 9/100
52/52 [==============================] - 1s 13ms/step - loss: nan - accuracy: 0.9968
Epoch 10/100
52/52 [==============================] - 1s 12ms/step - loss: nan - accuracy: 0.9968
Epoch 11/100
52/52 [=========

hasil akhir percobaan ke-10 nilai loss masih tetap nan dan akurasi masih 0.99.

hasil yang didapatkan adalah model belum dapat memprediksi val loss dan accuray yang baik. Ini dikarenakan mungkin masih terdapat data yang overfit sehingga setelah dinormalisasi dan dropout layer pun masih menghasilkan model yang kurang bagus

compile ann

In [ ]:
annmodelrework.compile(optimizer= 'adam', loss= 'binary_crossentropy', metrics= ['accuracy'])

# 1e. performa analisis

In [ ]:
y_pred = annmodel.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

26/26 [==============================] - 0s 1ms/step
[[0. 0.]
 [0. 0.]
 [0. 0.]
 ...
 [0. 0.]
 [0. 0.]
 [0. 0.]]


In [ ]:
#print confusion matrix and classification report
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[825   0]
 [  5   0]]
              precision    recall  f1-score   support

         0.0       0.99      1.00      1.00       825
         1.0       0.00      0.00      0.00         5

    accuracy                           0.99       830
   macro avg       0.50      0.50      0.50       830
weighted avg       0.99      0.99      0.99       830



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[825   0]
 [  5   0]]


0.9939759036144579

analisis:
performa dari arsitektur yang terdapat pada 1e dapat dikatakan belum memiliki performa yang baik sama juga dengan arstiktur pada 1d. Keduanya mempunyai val loss nan dan accuracy yang tinggi. Masih terdapat overfitting problem yang tidak terdeteksi. Penambahan dropout layer dan batch normalization sepertinya belum mampu untuk meingkatkan performa dari model